# Training
(WiP)

In [1]:
import graph_class as gc
import numpy as np
import pandas as pd
import dgl
from dgl.nn import GraphConv
import torch
from torch import nn


Using backend: pytorch


In [2]:
dataset = gc.WeatherDataset('test_one')
dataset.create('../data/data_initial_preprocessing.csv')

In [3]:
g = dataset.graph
g = dgl.add_self_loop(g)

In [4]:
g

Graph(num_nodes=667, num_edges=3540,
      ndata_schemes={'x': Scheme(shape=(1393, 7), dtype=torch.float64), 'y': Scheme(shape=(1393,), dtype=torch.float64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [5]:
#Classe da Rede Neural
class CGN(nn.Module):
    def __init__(self, in_feats, num_classes):
        super(CGN, self).__init__()
        self.conv1 = GraphConv(in_feats, 32, norm='both')
        self.conv2 = GraphConv(32, 16, norm='both')
        self.conv3 = GraphConv(16, num_classes, norm='both')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = torch.tanh(h)
        h = self.conv2(g, h)
        h = torch.tanh(h)
        h = self.conv3(g, h)

        return h

In [6]:
net = CGN(g.ndata['x'].shape[2], 1)

In [7]:
net = net.float()
net

CGN(
  (conv1): GraphConv(in=7, out=32, normalization=both, activation=None)
  (conv2): GraphConv(in=32, out=16, normalization=both, activation=None)
  (conv3): GraphConv(in=16, out=1, normalization=both, activation=None)
)

In [8]:
from torch import optim
from torch import nn
from torch.utils.tensorboard import SummaryWriter

In [9]:
p = {
    'epochs': 10000,
    'optim': optim.Adam,
    'loss_function': nn.MSELoss(),
    'lr': 1e-3
}

net = net.to('cuda')
g = g.to('cuda')
name = 'runs/test_3'

In [10]:
writer = SummaryWriter(f"{name}")

features = g.ndata['x'].float()
label = g.ndata['y'].float()

train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']

loss_fn = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=p['lr'])

history = {
    'loss': [],
    'val_loss': []
}

for epoch in range(p['epochs']):
    prediction = net(g, features)
    prediction = prediction.reshape(prediction.shape[0], prediction.shape[1])

    loss = loss_fn(prediction[train_mask], label[train_mask])
    val_loss = loss_fn(prediction[val_mask], label[val_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    history['loss'].append(loss.cpu().detach().numpy())
    history['val_loss'].append(val_loss.cpu().detach().numpy())

    writer.add_scalar("Loss/train", loss, epoch) #tensorboard
    writer.add_scalar("Loss/Val", val_loss, epoch) #tensorboard

    if epoch % 100 == 0:
        print(f'Epoch: {epoch} Loss: {loss} Val Loss: {val_loss}')
print(f'Epoch: {epoch} Loss: {loss} Val Loss: {val_loss}')

Epoch: 0 Loss: 1.0068614482879639 Val Loss: 0.9212799072265625
Epoch: 100 Loss: 0.9448891282081604 Val Loss: 0.8730277419090271
Epoch: 200 Loss: 0.9438853859901428 Val Loss: 0.8722531795501709
Epoch: 300 Loss: 0.9432968497276306 Val Loss: 0.871872067451477
Epoch: 400 Loss: 0.9427592754364014 Val Loss: 0.8715268969535828
Epoch: 500 Loss: 0.9421452879905701 Val Loss: 0.8711426854133606
Epoch: 600 Loss: 0.9413684606552124 Val Loss: 0.8706011772155762
Epoch: 700 Loss: 0.9403302073478699 Val Loss: 0.8698210120201111
Epoch: 800 Loss: 0.9389535188674927 Val Loss: 0.86882483959198
Epoch: 900 Loss: 0.9372388124465942 Val Loss: 0.8677184581756592
Epoch: 1000 Loss: 0.9351782202720642 Val Loss: 0.8665796518325806
Epoch: 1100 Loss: 0.9327078461647034 Val Loss: 0.8657634854316711
Epoch: 1200 Loss: 0.9298631548881531 Val Loss: 0.8651872873306274
Epoch: 1300 Loss: 0.9268057346343994 Val Loss: 0.8642699718475342
Epoch: 1400 Loss: 0.9233274459838867 Val Loss: 0.862850546836853
Epoch: 1500 Loss: 0.920026

In [19]:
dataset.scaler_y.inverse_transform(np.array([0.94327]).reshape(-1,1))

ValueError: non-broadcastable output operand with shape (1,1) doesn't match the broadcast shape (1,1393)

In [22]:
label[train_mask].shape

torch.Size([533, 1400])

In [23]:
dataset.scaler_y.inverse_transform(np.repeat(np.array(1.069), 1400).reshape(-1, 1400))

array([[11.39190968,  7.16351918,  5.75264139, ..., 15.19342615,
        15.11701595, 15.01706286]])

In [15]:
from sklearn.preprocessing import StandardScaler

s = StandardScaler()
teste = np.array([1,2,1,4,8,1,0,4,-2]).reshape(-1,1)
s.fit(teste)

StandardScaler()

In [17]:
s.transform(np.array([9]).reshape(-1,1))

array([[2.52693135]])